## **Import Libraries**

In [51]:
import openai
import chromadb
from chromadb.utils import embedding_functions
import pandas as pd
from config import API
import warnings
warnings.filterwarnings("ignore")

## **Text Embedding Function**

In [52]:
def text_embedding(text):
    response = openai.Embedding.create(model="text-embedding-ada-002", input = text)
    return response["data"][0]["embedding"]
 
openai_ef = embedding_functions.OpenAIEmbeddingFunction(api_key=API.OPENAI_KEY, model_name="text-embedding-ada-002")

## **RAG Disease Datas**

In [53]:
with open("data/disease_data.txt", "r") as f:
    disease_str_data = f.read()
disease_str_data = disease_str_data.replace("\n", " ")
disease_str_data = disease_str_data.replace("===", "\n")
disease_data = {"text": disease_str_data.split("\n")}
disease_df = pd.DataFrame(disease_data)
disease_df.head()

,text
0,# 생식기호흡증후군 모돈의 번식장애와 자돈에는 호흡기 증세가 특징인 질병으로 199...
1,# 돈단독 돈단독은 돈단독균의 감염에 의한 돼지의 급성 및 만성 감염병으로서 급...
2,# 생식기호흡증후군 모돈의 번식장애와 자돈에는 호흡기 증세가 특징인 질병으로 1...
3,# 오제스키병 오제스키병 허피스바이러스 감염에 의한 감염병으로 열을 동반한 신...


In [79]:
client = chromadb.PersistentClient(path="DB")
disease_collection = client.get_or_create_collection("disease", embedding_function=openai_ef)

docs = disease_df["text"].tolist()
ids = [str(x) for x in disease_df.index.tolist()]

disease_collection.add(documents=docs, ids=ids)

Add of existing embedding ID: 0
Add of existing embedding ID: 1
Add of existing embedding ID: 2
Add of existing embedding ID: 3
Insert of existing embedding ID: 0
Insert of existing embedding ID: 1
Insert of existing embedding ID: 2
Insert of existing embedding ID: 3


In [55]:
vector = text_embedding("돼지에게 구토 증상이 나타나.")

results = disease_collection.query(    
    query_embeddings=vector,
    n_results=5,
    include=["documents"]
)

res = "\n".join(str(item) for item in results["documents"][0])

print(res)

Number of requested results 5 is greater than number of elements in index 4, updating n_results = 4


   # 오제스키병 오제스키병 허피스바이러스 감염에 의한 감염병으로 열을 동반한 신경증상과 호흡기증상이 있는 것이 특징입니다. 어린 돼지가 감염되면 대부분 폐사하며 임신돈이 감염되면 사산과 유산을 하는 것이 특징인 우리나라 제 1종 법정가축전염병입니다 병인론: 허피스 바이러스 (Herpes virus)에 속하는 오제스키병바이러스   # 역학 1. 헝가리의 오제스키에 의해 1902년에 처음 보고됨 2. 전세계적 발병. 우리나라도 1987년 발생보고된 이후 경기도를 중심으로 확산되고 있는 실정 3. 우리나라 제 1종 법정가축전염병 4. 직사광선, 열, 햇빛, 일반소독제에 사멸, 소독제로는 요오드포르계(바이오시드30), 치아염소산소다, 페놀, 생석회 5. PH 6-8에서 바이러스가 잘 생존 그 이외의 범위에서 잘 생존하지 못함 6. 온도의 변화가 심할 때 바이러스가 잘 생존하지 못함 7. 영상 4도이하에서 잘 생존.돼지가 진정한 자연숙주; 다른 동물(말, 당나귀 제외)도 감염되기는 하지만 대부분 폐사하고 전파원이 되지 못함 8. 대부분 감염돈 또는 보독돈(감염회복돈)의 입식에 의해 농장에 전파 9. 야생동물(특히 설치류, 너구리)에 의해 농장에서 농장으로 전파 10. 돼지에 잠복감염 →  외부 임상증상이 없더라도 재발하거나 바이러스를 배출  # 경구 기전 또는 경비감염 → 상부호흡기에 감염, 증식 →  후각신경의 축삭을 따라 중추신경계로 이동 → 호흡기를 따라 하강하여 기관지, 폐포에 감염 → 중추신경계, 호흡기계에서 감염, 증식하던 바이러스가 바이러스혈증을 일으킴 → 다른 조직감염(생식기계 감염) 임상증상: 감염되는 연령에 따라 다양. 질병의 심한 정도도 돈군에 따라 다양하다  포유자돈  - 급사, 높은 이환율, 폐사율은 100%에 이름, 특징적인 임상증상 없이 폐사 - 고열(열혈증), 침울, 진전, 협조운동불능, 경련 직후에 폐사 - 구토, 설사 또는 변비 이유자돈 및 육성돈  - 폐렴, 급성 건성 기침, 높은 이환율, 감기와 같은 증상(flu-like synd

## **RAG Medicine Data**

In [57]:
with open("data/medicine_data.txt", "r") as f:
    medicine_str_data = f.read()
medicine_str_data = medicine_str_data.replace("\n", " ")
medicine_str_data = medicine_str_data.replace(r"====== ", "\n")
medicine_str_data = medicine_str_data.replace("～", "~")
medicine_data = {"text": medicine_str_data.split("\n")}
medicine_df = pd.DataFrame(medicine_data)
medicine_df.head()

,text
0,# 기본 정보 - 제품명: 골든비.에치.티 - 제품 영문명: GOLDEN B.H.T...
1,# 기본 정보 - 제품명: 골든에독시퀸 - 제품 영문명: GOLDEN ETHOXY...
2,# 기본 정보 - 제품명: 주문용배합사료첨가제 - 업체명: (주)우성양행 - 허가...
3,# 기본 정보 - 제품명: 비타린칼 펠렛 - 제품 영문명: Vita-Rinkal ...
4,# 기본 정보 - 제품명: 응애캐치 액 - 제품 영문명: EungAe Catch ...


In [60]:
client = chromadb.PersistentClient(path="DB")
medicine_collection = client.get_or_create_collection("medicine", embedding_function=openai_ef)

docs = medicine_df["text"].tolist()
ids = [str(x) for x in medicine_df.index.tolist()]

medicine_collection.add(documents=docs, ids=ids)

Insert of existing embedding ID: 0
Insert of existing embedding ID: 1
Insert of existing embedding ID: 2
Insert of existing embedding ID: 3
Insert of existing embedding ID: 4
Insert of existing embedding ID: 5
Insert of existing embedding ID: 6
Insert of existing embedding ID: 7
Insert of existing embedding ID: 8
Insert of existing embedding ID: 9
Insert of existing embedding ID: 10
Insert of existing embedding ID: 11
Insert of existing embedding ID: 12
Insert of existing embedding ID: 13
Insert of existing embedding ID: 14
Insert of existing embedding ID: 15
Insert of existing embedding ID: 16
Insert of existing embedding ID: 17
Insert of existing embedding ID: 18
Insert of existing embedding ID: 19
Insert of existing embedding ID: 20
Insert of existing embedding ID: 21
Insert of existing embedding ID: 22
Insert of existing embedding ID: 23
Insert of existing embedding ID: 24
Insert of existing embedding ID: 25
Insert of existing embedding ID: 26
Insert of existing embedding ID: 27
In

In [61]:
vector = text_embedding("돼지에게 구토 증상이 나타나.")

results = medicine_collection.query(    
    query_embeddings=vector,
    n_results=5,
    include=["documents"]
)

res = "\n".join(str(item) for item in results["documents"][0])

print(res)

 # 주의사항 가. 다음 돼지에게는 투여하지 말 것  (1) 본제에 대한 과민반응이나 쇼크를 일으킨 일이 있었던 돼지에게는       사용하지 마십시오.  (2) 열이 있거나 영양장애가 심한 돼지, 전염성 질병 및 기생충 감염,       스트레스를 받은 돼지,       곰팡이 및 세균의 독소 등(아플라톡신, 대장균 및 살모넬라 등)에 의해       면역 기능이 저하된 돼지에게는 사용하지 마십시오. 나. 부작용  - 개체에 따라 간혹 백신접종 후에 식욕부진, 허탈, 피부발진, 구토,     경련 등의 과민반응이 일어날 수 있으므로     주사 후 과민반응의 발현여부를 잘 관찰하시고 과민반응이 나타났을 때에는    수의사와 상담하여 충분한 마사지와 에피네프린 투여 등     적절한 처치를 하십시오. 다. 일반적 주의사항  (1) 반드시 수의사의 지시에 따라 사용하십시오.  (2) 건강한 돼지에만 사용하십시오.  (3) 정해진 용법, 용량을 준수하십시오.  (4) 지정된 축종 이외에는 사용하지 마십시오.  (5) 냉장고에서 바로 꺼내어 접종하면 과민반응이 일어날 수 있으므로       백신을 동물 체온과 비슷한 온도로 가온한 후 사용하십시오.  (6) 백신접종 전에는 반드시 백신의 성상 및 이물의 혼입여부 등을 관찰한 후       이상이 있으면 사용하지 마십시오.  (7) 얼었다 녹은 제품은 품질 변화 또는 효력이 저하되므로 사용하지 마십시오  (8) 다른 제제(백신, 항생제, 영양제 등)와 혼합하여 사용하지 마십시오.  (9) 개봉한 백신은 개봉 즉시 모두 사용하십시오.  (10) 접종 후 최소한 3-5일간은 돼지를 안정시키십시오. 라. 적용상의 주의사항  (1) 반드시 근육주사 하십시오.  (2) 피하 접종시에는 경결 및 화농이 생길 수 있으며       혈관내로 주입시에는 쇼크 및 폐사가 일어날 수 있습니다.  (3) 사용할 때마다 충분히 흔들어서 균등하게 한 후 사용하십시오.  (4) 멸균된 주사기와 주사침을 사용하며 1두 1침을 원칙으로 사용하십시오